In [101]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 데이터 경로 설정
data_dir = r'C:\Users\billn\Desktop\GAFT\CNN DATA\train'  # train 폴더 경로 확인 필요

# 이미지 데이터 생성기
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 데이터 정규화 및 검증 세트 분할

# 훈련 데이터 생성기
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=20,
    class_mode='binary',
    subset='training'
)

# 검증 데이터 생성기
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=20,
    class_mode='binary',
    subset='validation'
)

# 모델 구성
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # 한 에포크에 사용할 스텝 수
    epochs=10,  # 반복 횟수
    validation_data=validation_generator,
    validation_steps=50  # 검증 스텝 수
)

# 모델 저장
model.save('C:/Users/billn/Desktop/GAFT/CNN DATA/model.h5')


Found 116 images belonging to 2 classes.


Found 28 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 1s 7ms/step - loss: 1.1972 - accuracy: 0.5000 - val_loss: 0.7606 - val_accuracy: 0.5000


In [102]:
from tensorflow.keras.models import load_model

# 모델 불러오기
model = load_model('C:/Users/billn/Desktop/GAFT/CNN DATA/model.h5')


In [103]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 폴더 경로
img_folder_path = 'C:/Users/billn/Desktop/GAFT/CNN DATA/testimg'

# 폴더 내의 모든 이미지 파일 불러오기
for img_filename in os.listdir(img_folder_path):
    img_path = os.path.join(img_folder_path, img_filename)
    print("Processing:", img_path)

    # 이미지 불러오기 및 전처리
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델 입력 형태에 맞게 차원 추가
    img_array /= 255.0  # 이미지 데이터 정규화

    # 모델 예측
    predictions = model.predict(img_array)
    print("Predictions for", img_filename, ":", predictions)


Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.1.png
1/1 [==============================] - 0s 51ms/step
Predictions for abnormal.1.png : [[0.66358036]]
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.10.png
1/1 [==============================] - 0s 18ms/step
Predictions for abnormal.10.png : [[0.66110927]]
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.11.png
1/1 [==============================] - 0s 17ms/step
Predictions for abnormal.11.png : [[0.69372535]]
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.12.png
1/1 [==============================] - 0s 17ms/step
Predictions for abnormal.12.png : [[0.6952228]]
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.13.png
1/1 [==============================] - 0s 17ms/step
Predictions for abnormal.13.png : [[0.7050997]]
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.14.png
1/1 [==============================] - 0s 18ms/step
Predictio

In [104]:
threshold = 0.5  # 결정 임계값 설정
class_labels = ['normal', 'abnormal']  # 클래스 레이블

for img_filename, prediction in zip(os.listdir(img_folder_path), predictions):
    label_index = 1 if prediction >= threshold else 0
    print(f"{img_filename}: {class_labels[label_index]} ({prediction[0]*100:.2f}%)")


abnormal.1.png: abnormal (68.59%)


In [105]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 폴더 경로
img_folder_path = 'C:/Users/billn/Desktop/GAFT/CNN DATA/testimg'

# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('C:/Users/billn/Desktop/GAFT/CNN DATA/model.h5')

# 예측 결과를 저장할 리스트
predictions = []

# 폴더 내의 모든 이미지 파일 불러오기
for img_filename in os.listdir(img_folder_path):
    img_path = os.path.join(img_folder_path, img_filename)
    print("Processing:", img_path)

    # 이미지 불러오기 및 전처리
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델의 입력으로 맞춤
    img_array /= 255.0  # 이미지 데이터 정규화

    # 모델 예측
    prediction = model.predict(img_array)
    predictions.append(prediction[0])

# 임계값과 레이블 설정
threshold = 0.5
class_labels = ['normal', 'abnormal']

# 모든 이미지 파일에 대한 예측 결과 출력
for img_filename, prediction in zip(os.listdir(img_folder_path), predictions):
    label_index = 1 if prediction >= threshold else 0
    print(f"{img_filename}: {class_labels[label_index]} ({prediction[0]*100:.2f}%)")


Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.1.png
1/1 [==============================] - 0s 49ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.10.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.11.png
1/1 [==============================] - 0s 18ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.12.png
1/1 [==============================] - 0s 18ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.13.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.14.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.15.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\abnormal.16.png
1/1 [==============================] - 0s 17ms/step
P

In [107]:
import os

train_normal_dir = os.path.join('C:/Users/billn/Desktop/GAFT/CNN DATA/train', 'normal')
train_abnormal_dir = os.path.join('C:/Users/billn/Desktop/GAFT/CNN DATA/train', 'abnormal')

print('Total training normal images:', len(os.listdir(train_normal_dir)))
print('Total training abnormal images:', len(os.listdir(train_abnormal_dir)))


Total training normal images: 72
Total training abnormal images: 72


In [108]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001),  # 학습률 조정
              loss='binary_crossentropy', 
              metrics=['accuracy'])


In [109]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 검증용 데이터 분할
)


In [110]:
# 데이터 경로 설정
train_dir = r'C:\Users\billn\Desktop\GAFT\CNN DATA\train'  # 훈련 데이터 폴더 경로

# 훈련 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),  # 이미지 크기 조정
    batch_size=32,  # 한 번에 처리할 이미지 수
    class_mode='binary',  # 또는 'categorical' if one-hot encoded
    subset='training'  # 훈련 데이터
)

# 검증 데이터 생성기
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # 검증 데이터
)


Found 116 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [111]:
from tensorflow.keras.layers import Dropout

# 기존 모델에 드롭아웃 추가
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련 과정에서 검증 데이터를 사용하여 오버피팅을 모니터링
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20
)


Epoch 1/20
4/4 [==============================] - 2s 294ms/step - loss: 1.1187 - accuracy: 0.5259 - val_loss: 0.6957 - val_accuracy: 0.5000
Epoch 2/20
4/4 [==============================] - 1s 192ms/step - loss: 0.7185 - accuracy: 0.4828 - val_loss: 0.6881 - val_accuracy: 0.5000
Epoch 3/20
4/4 [==============================] - 1s 196ms/step - loss: 0.7065 - accuracy: 0.5431 - val_loss: 0.6842 - val_accuracy: 0.5000
Epoch 4/20
4/4 [==============================] - 1s 230ms/step - loss: 0.6768 - accuracy: 0.5345 - val_loss: 0.6854 - val_accuracy: 0.5000
Epoch 5/20
4/4 [==============================] - 1s 227ms/step - loss: 0.6877 - accuracy: 0.4828 - val_loss: 0.6882 - val_accuracy: 0.6071
Epoch 6/20
4/4 [==============================] - 1s 197ms/step - loss: 0.6922 - accuracy: 0.4397 - val_loss: 0.6880 - val_accuracy: 0.7143
Epoch 7/20
4/4 [==============================] - 1s 200ms/step - loss: 0.6910 - accuracy: 0.5862 - val_loss: 0.6893 - val_accuracy: 0.7143
Epoch 8/20
4/4 [====

In [116]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지 폴더 경로
img_folder_path = 'C:/Users/billn/Desktop/GAFT/CNN DATA/testimg'

# 모델 불러오기
from tensorflow.keras.models import load_model
model = load_model('C:/Users/billn/Desktop/GAFT/CNN DATA/model.h5')

# 예측 결과를 저장할 리스트
predictions = []

# 폴더 내의 모든 이미지 파일 불러오기
for img_filename in os.listdir(img_folder_path):
    img_path = os.path.join(img_folder_path, img_filename)
    print("Processing:", img_path)

    # 이미지 불러오기 및 전처리
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델의 입력으로 맞춤
    img_array /= 255.0  # 이미지 데이터 정규화

    # 모델 예측
    prediction = model.predict(img_array)
    predictions.append(prediction[0])

# 임계값과 레이블 설정
threshold = 0.5
class_labels = ['normal', 'abnormal']

# 모든 이미지 파일에 대한 예측 결과 출력
for img_filename, prediction in zip(os.listdir(img_folder_path), predictions):
    label_index = 1 if prediction >= threshold else 0
    print(f"{img_filename}: {class_labels[label_index]} ({prediction[0]*100:.2f}%)")


Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.1 - 복사본 (2).png
1/1 [==============================] - 0s 50ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.1 - 복사본.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.1.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.10 - 복사본 (2).png
1/1 [==============================] - 0s 18ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.10 - 복사본.png
1/1 [==============================] - 0s 19ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.10.png
1/1 [==============================] - 0s 17ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.11 - 복사본 (2).png
1/1 [==============================] - 0s 18ms/step
Processing: C:/Users/billn/Desktop/GAFT/CNN DATA/testimg\normal.11 - 복사본.png
1/1 [==================

In [113]:
import os

train_normal_dir = os.path.join('C:/Users/billn/Desktop/GAFT/CNN DATA/train', 'normal')
train_abnormal_dir = os.path.join('C:/Users/billn/Desktop/GAFT/CNN DATA/train', 'abnormal')

num_normal = len(os.listdir(train_normal_dir))
num_abnormal = len(os.listdir(train_abnormal_dir))

print(f"Total training normal images: {num_normal}")
print(f"Total training abnormal images: {num_abnormal}")

# 데이터 불균형 확인 후 조정
if num_normal != num_abnormal:
    print("Data is imbalanced. Consider using augmentation techniques or resampling the dataset.")


Total training normal images: 72
Total training abnormal images: 72
